## Voronoi HBEC


In [13]:
# %% 
import numpy as np
import matplotlib.pyplot as plt
import pickle
import cv2
from cellpose import plot
import os
from skimage.measure import label, regionprops, regionprops_table
from skimage.color import label2rgb
import pandas as pd
import glob
import sys

sys.path.append('../vasco_scripts')  # add the relative path to the folder
from defects import *  # import the module from the folder

import pathlib
import seaborn as sns
from pip._internal.cli.progress_bars import get_download_progress_renderer
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import sys,time,random
def progressBar(count_value, total, suffix=''):
    bar_length = 100
    filled_up_Length = int(round(bar_length* count_value / float(total)))
    percentage = round(100.0 * count_value/float(total),1)
    bar = '=' * filled_up_Length + '-' * (bar_length - filled_up_Length)
    sys.stdout.write('[%s] %s%s ...%s\r' %(bar, percentage, '%', suffix))
    sys.stdout.flush()

def find_centers(masks):
    '''find centers in segmented image b/w or labels'''
    regions = regionprops(masks)
    x_cent = []
    y_cent = []
    label_num = []
    for i,props in enumerate(regions):
        yi, xi = props.centroid
        x_cent.append(xi)
        y_cent.append(yi)
        label_num.append(props.label)
        # plt.text(xi, yi, str(i))
    return x_cent, y_cent, label_num

def center_pairs(Xlong, Xshort):
    '''find indexes of Xshort in Xlong'''
    from scipy import spatial
    tree = spatial.KDTree(Xlong)
    _, minid = tree.query(Xshort)   
    return minid


def brightness(image, mask, labels):
    '''measure raw image intenslity for each mask label'''
    intensity = []
    for l in labels:
        intensity.append(image[mask==l].mean())
        progressBar(l,len(labels))
        
    return intensity 


In [14]:
# %%
%matplotlib qt

# folder = r"D:\HBEC\s2_250_500"
folder = r"C:\Users\victo\Downloads\SB_lab\HBEC\s2(120-919)"
raw_list = sorted(glob.glob(folder + "/*.tif"))
mask_list = []
for raw in raw_list:
    # mask_folder = os.path.dirname(raw) + "/masks/"
    # mask_name = os.path.basename(raw).split('.')[0] + "_seg.npy"
    # mask_list.append(mask_folder + mask_name)
    mask_path = os.path.join(
        str(pathlib.Path(raw).parents[0]), 
        "Mask", 
        raw.split(os.sep)[-1].split(".")[0]+"_cp_masks.png"
        )    
    mask_list.append(mask_path)



# %% TEST CELL CENTERS AND BOUNDARIES WITH VORONOI
# NEXT: COUNT NEIGHBOUR NUMBER FOR EACH CELL
from scipy.spatial import Voronoi, voronoi_plot_2d

colors = plt.cm.tab10(np.arange(10))

fig, ax = plt.subplots(1,1, figsize=(8,8))
# dw=-1 will take entire image
h, w, dw = 200, 600, 500 
num = 200
dt = 1
for (i,praw), pmask in zip(enumerate(raw_list[num::dt]),mask_list[num::dt]):
    print(i, praw,"\n", pmask)
    if dw==-1:
        img = cv2.imread(praw)[:,:,0]
        mask = cv2.imread(pmask, flags=cv2.IMREAD_ANYDEPTH)    
    else: 
        img = cv2.imread(praw)[h:h+dw,w:w+dw,0]
        mask = cv2.imread(pmask, flags=cv2.IMREAD_ANYDEPTH)[h:h+dw,w:w+dw]
    if i==0:
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        img_clahe = clahe.apply(img)
        ax.imshow(255-img_clahe, "gray")         
        # ax.imshow(img)
        # ax.imshow(label2rgb(mask, bg_label=0), alpha=0.2)

    x_cent, y_cent,_ = find_centers(mask)
    ax.plot(x_cent, y_cent, 'o', color=colors[i], alpha=.3) 

    points = np.vstack((x_cent, y_cent)).T
    vor = Voronoi(points)
    voronoi_plot_2d(vor, ax=ax, show_vertices=False, line_colors="red", line_width=2, line_alpha=0.5, point_size=2)


    ax.set_xlim([0,img.shape[0]])
    ax.set_ylim([0,img.shape[0]])
    break
    if i==1:
        break

0 C:\Users\victo\Downloads\SB_lab\HBEC\s2(120-919)\Trans__201.tif 
 C:\Users\victo\Downloads\SB_lab\HBEC\s2(120-919)\Mask\Trans__201_cp_masks.png
